### Model training

The filtered, calculated and encoded features can now be trained with appropriate models.

The following approaches are considered:
- Multi classification problem with clustered data
- Mutli regression problem with two outputs (longitude/latitude)

In terms of the data we have the following approaches:
- variable sequence length - can take all points in POLYLINE in consideration, zero-padd start of sequence if necessary
- fixed sequence length - take 5 points from beginning of POLYLINE and 5 points from end of polyline 


Algorithms:
- Long term short term NN (multi-class classification and regression)
    - able to handle variable sequence length, therefore the total trip POLYLINE can be used
- Random forest(only regression) 
    - can handle outliers well as dataset still contains outliers
    - runs efficiently on large data set

In [ ]:
import tensorflow

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense 
from tensorflow.keras.optimizers import Adam

In [ ]:
import pandas as pd 
import numpy as np 
import json

In [ ]:
train_data = pd.read_csv('s3://think-tank-casestudy/features_engineered/feature_engineered_regress_train.csv')

In [ ]:
def polyline_to_sequences(data, n_start, n_end):
    sequence_start = data.POLYLINE_LIST.apply(lambda sequence: json.loads(sequence)[:n_start])
    sequence_end = data.POLYLINE_LIST.apply(lambda sequence: json.loads(sequence)[-n_end+1:-1])
    return sequence_start, sequence_end

In [ ]:
X = train_data.iloc[:,3:-2]
y = train_data[[['DEST_LON','DEST_LAT']]

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
rf = RandomForestRegressor(random_state=0)
wrapper = MultiOutputRegressor(rf)
cv = RepeatedKFold(n_splits=10,random_state=1)
n_scores = cross_val_score(wrapper, X, y, scoring='mean_squared_error', cv=cv)

In [ ]:
model = Sequential()
model.add(Input(input_shape=(X.shape[0], X.shape[1])))
model.add(LSTM(50, activation='relu'))
model.add(Dense(2), activation='linear')

model.compile(optimizer='adam', loss='mse', metrics=['MeanSquaredError', 'MeanAbsoluteError'])
model.add(Dense(n_outputs))
X_pred_train = model.fit(X_train)